# Çok Değişkenli Aykiri Gözlem Analizi Algoritması

## 1) Veri setinde sayısal değişkenleri ayrıştırın.

## 2) Veri setindeki her değişkenin aykırı değerlerini tanımlayın. Bunları tespit etmek için farklı yöntemler kullanılabilir. Örneğin boxplot, histogram, scatterplot gibi grafiksel yöntemler kullanılabilir.

## 3) Aykırı değerleri baskılama yöntemiyle düzeltmek için bir model seçin. Bu, örneğin çoklu doğrusal regresyon, lojistik regresyon veya diğer modeller olabilir. Bu modele göre, aykırı gözlemlerin tahmini değerleri hesaplanır.

## 4) Aykırı gözlemlerin değerlerini, bu tahmini değerlerle değiştirin. Bu, aykırı değerlerin neden olduğu sapmayı azaltacaktır.

## 5) Değiştirilen veri seti üzerinde analizleri yapın. Bu şekilde, aykırı gözlemlerin neden olduğu yanlış sonuçlardan kurtabilirsiniz.

# ----------------------------------------------------

# 1) Veri setinin import edilmesi ve sayısal değişkenlerin ayrıştırılması

In [1]:
import seaborn as sns
import pandas as pd

In [2]:
df = sns.load_dataset('diamonds')
df = df.select_dtypes(include=['float64', 'int64'])
df = df.dropna()
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


## 2) Veri setindeki aykırı değerlerin tespiti

In [3]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [5]:
model = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)

In [6]:
model.fit_predict(df)

array([-1, -1, -1, ...,  1,  1,  1])

In [7]:
df_scores = model.negative_outlier_factor_

In [8]:
df_scores[0:10]

array([-1.58352526, -1.59732899, -1.62278873, -1.33002541, -1.30712521,
       -1.28408436, -1.28428162, -1.26458706, -1.28422952, -1.27351342])

In [9]:
np.sort(df_scores)[0:20]

array([-8.60430658, -8.20889984, -5.86084355, -4.98415175, -4.81502092,
       -4.81502092, -4.61522833, -4.37081214, -4.29842288, -4.10492387,
       -4.0566648 , -4.01831733, -3.94882806, -3.82378797, -3.80135297,
       -3.75680919, -3.65947378, -3.59249261, -3.55564138, -3.47157375])

In [12]:
esik_deger = np.sort(df_scores)[13]
esik_deger

-3.823787967755565

In [14]:
aykiri_olmayan = df_scores > esik_deger
aykiri_olmayan

array([ True,  True,  True, ...,  True,  True,  True])

In [20]:
~aykiri_olmayan
# Aykiri Gözlemler

array([False, False, False, ..., False, False, False])

# 3) Aykırı gözlemleri veri setine entegre etme

In [17]:
# Eşik değerin veri setindeki karşılığı olan gözlemi elde etme

baski_deger = df[df_scores == esik_deger]
baski_deger

,carat,depth,table,price,x,y,z
31230,0.45,68.6,57.0,756,4.73,4.5,3.19


In [22]:
aykiri_degerler = df[~aykiri_olmayan]
aykiri_degerler.head()

,carat,depth,table,price,x,y,z
6341,1.00,44.0,53.0,4032,6.31,6.24,4.12
10377,1.09,43.0,54.0,4778,6.53,6.55,4.12
24067,2.00,58.9,57.0,12210,8.09,58.90,8.06
31230,0.45,68.6,57.0,756,4.73,4.50,3.19
35633,0.29,62.8,44.0,474,4.20,4.24,2.65


In [30]:
aykiri_degerler.to_records(index=False)
# aykiri değerler array türüne çevrildi.

rec.array([(1.  , 44. , 53. ,  4032, 6.31,  6.24,  4.12),
           (1.09, 43. , 54. ,  4778, 6.53,  6.55,  4.12),
           (2.  , 58.9, 57. , 12210, 8.09, 58.9 ,  8.06),
           (0.45, 68.6, 57. ,   756, 4.73,  4.5 ,  3.19),
           (0.29, 62.8, 44. ,   474, 4.2 ,  4.24,  2.65),
           (0.3 , 51. , 67. ,   945, 4.67,  4.62,  2.37),
           (0.73, 70.8, 55. ,  1049, 5.51,  5.34,  3.84),
           (1.03, 78.2, 54. ,  1262, 5.72,  5.59,  4.42),
           (0.7 , 71.6, 55. ,  1696, 5.47,  5.28,  3.85),
           (0.51, 61.8, 54.7,  1970, 5.12,  5.15, 31.8 ),
           (0.51, 61.8, 55. ,  2075, 5.15, 31.8 ,  5.12),
           (0.81, 68.8, 79. ,  2301, 5.26,  5.2 ,  3.58),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [31]:
res = aykiri_degerler.to_records(index=False)
# aykiri değerler array türüne çevrildi.

In [32]:
res[:] = baski_deger.to_records(index=False)
res

# aykırı verilerin değerleri eşik değerin verileri ile değiştirildi!

rec.array([(0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [33]:
df[~aykiri_olmayan] = pd.DataFrame(res, index= df[~aykiri_olmayan].index)
# aykiri verilerin yeni değerleri orjinal veri setine olduğu gibi entegre edildi.

In [29]:
df[~aykiri_olmayan]

,carat,depth,table,price,x,y,z
6341,0.45,68.6,57.0,756,4.73,4.5,3.19
10377,0.45,68.6,57.0,756,4.73,4.5,3.19
24067,0.45,68.6,57.0,756,4.73,4.5,3.19
31230,0.45,68.6,57.0,756,4.73,4.5,3.19
35633,0.45,68.6,57.0,756,4.73,4.5,3.19
36503,0.45,68.6,57.0,756,4.73,4.5,3.19
38840,0.45,68.6,57.0,756,4.73,4.5,3.19
41918,0.45,68.6,57.0,756,4.73,4.5,3.19
45688,0.45,68.6,57.0,756,4.73,4.5,3.19
48410,0.45,68.6,57.0,756,4.73,4.5,3.19


# -------------------------------------------------------

In [34]:
import seaborn as sns
import pandas as pd

In [35]:
df = sns.load_dataset('tips')
df = df.select_dtypes(include=['float64', 'int64'])
df = df.dropna()
df.head()

,total_bill,tip,size
0,16.99,1.01,2
1,10.34,1.66,3
2,21.01,3.50,3
3,23.68,3.31,2
4,24.59,3.61,4


In [36]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [39]:
model = LocalOutlierFactor(n_neighbors=10, contamination=0.1)

In [40]:
model.fit_predict(df)

array([-1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1

In [41]:
df_scores = model.negative_outlier_factor_
df_scores[0:10]

array([-1.50224414, -1.4466781 , -1.02316662, -0.999761  , -1.02709145,
       -0.99640555, -1.2186794 , -0.99503439, -1.06979348, -1.1096861 ])

In [42]:
np.sort(df_scores)[0:10]

array([-2.85293963, -2.54821141, -2.36236654, -1.90287892, -1.64836245,
       -1.64140248, -1.64048888, -1.50224414, -1.47349993, -1.4466781 ])

In [46]:
esik_deger = np.sort(df_scores)[3]
esik_deger

-1.9028789212827248

In [47]:
aykiri_tf = df_scores > esik_deger
aykiri_tf

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [48]:
baski_deger = df[df_scores == esik_deger]
baski_deger

,total_bill,tip,size
232,11.61,3.39,2


In [49]:
aykirilar = df[~aykiri_tf]
aykirilar.head()

,total_bill,tip,size
67,3.07,1.00,1
172,7.25,5.15,2
178,9.60,4.00,2
232,11.61,3.39,2


In [50]:
aykirilar.to_records(index=False)

rec.array([( 3.07, 1.  , 1), ( 7.25, 5.15, 2), ( 9.6 , 4.  , 2),
           (11.61, 3.39, 2)],
          dtype=[('total_bill', '<f8'), ('tip', '<f8'), ('size', '<i8')])

In [51]:
res = aykirilar.to_records(index=False)

In [52]:
res[:] = baski_deger.to_records(index=False)
res

rec.array([(11.61, 3.39, 2), (11.61, 3.39, 2), (11.61, 3.39, 2),
           (11.61, 3.39, 2)],
          dtype=[('total_bill', '<f8'), ('tip', '<f8'), ('size', '<i8')])

In [53]:
df[~aykiri_tf] = pd.DataFrame(res, index = df[~aykiri_tf].index)
df[~aykiri_tf]

,total_bill,tip,size
67,11.61,3.39,2
172,11.61,3.39,2
178,11.61,3.39,2
232,11.61,3.39,2


# aykiri değerleri tespit et, aykiri değerleri ayrıştır ve array'e dönüştür, eşik değeri bul, aykiri verilerin değerlerini eşik değere eşitle, aykiri değerleri veri setine entegre et.